In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yaml
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Прочтём файл конфига с путями

CONFIG_PATH = "config.yaml"
with open(CONFIG_PATH, "r", encoding="utf-8") as config_file:
    CONFIG = yaml.load(config_file, Loader=yaml.FullLoader)

In [ ]:
# Загрузим необходимые датасеты

df = pd.read_csv(CONFIG['datasets_folder'] + '/data.csv')
post_df = pd.read_csv(CONFIG['datasets_folder'] + '/post_data.csv').rename(columns={'id': 'post_id'})

In [ ]:
df.head()

##### Обработаем признаки

In [ ]:
# Временные признаки - выделим из времени совершённого действия месяц, число, день недели и час

df["timestamp"] = pd.to_datetime(df["timestamp"])

df['month'] = df["timestamp"].dt.month.astype(int)
df['hour'] = df["timestamp"].dt.hour.astype(int)
df['day'] = df["timestamp"].dt.day.astype(int)
df['weekday'] = df["timestamp"].dt.weekday.astype(int)

In [ ]:
# Приведем некоторые категориальные признаки к бинарному виду

df['os'] = df['os'].map({'Android': 0, 'iOS': 1})
df['source'] = df['source'].map({'ads': 0, 'organic': 1})

In [ ]:
# Уберем колонку с текстом новости, т.к. обработка текста будет вестись через post_data.csv

df = df.drop(['text'], axis=1)

##### Выделим текстовые признаки

In [ ]:
post_df = pd.read_csv('../data' + '/post_data.csv').rename(columns={'id': 'post_id'})

post_df.head()

In [ ]:
# Уберём символ переноса строки

post_df['text'] = post_df['text'].str.replace('\n',' ')

In [ ]:
# Найдем tf-idf матрицу нашего корпуса текстов и выделим признаки

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
tfidf_df_ = tfidf.fit_transform(post_df['text'])

In [ ]:
tfidf_df = []

for row in tfidf_df_:
    tfidf_df.append([row.sum(), row.mean(), row.max()])

In [ ]:
tfidf_df = pd.DataFrame(tfidf_df, index=post_df['post_id'], columns=['tfidf_sum', 'tfidf_mean', 'tfidf_max']).reset_index()

In [ ]:
tfidf_df.head()

In [ ]:
# Получим финальный датасет, на котором будем обучать модель и сохраним вместе с tf-idf признаками

df = df.merge(tfidf_df, on='post_id')
df = df.sort_values(by='timestamp')

df.to_csv(CONFIG['datasets_folder'] + '/processed_df.csv', index=False)